# Exercicio do Keras

## Prever partido político com base nos votos

* Usaremos um conjunto de dados público sobre como congressistas dos EUA votaram em 17 questões diferentes no ano de 1984. Vamos ver se conseguimos determinar o partido político deles apenas com base nos votos, usando uma rede neural profunda.
* Para aqueles que não estão familiarizados com o sistema político dos EUA, os dois principais partidos são os "Democratas" e os "Republicanos". Atualmente, eles representam ideologias progressistas e conservadoras, respectivamente.
* Em 1984, a política não era tão polarizada quanto hoje, mas você ainda deve conseguir obter mais de 90% de precisão sem grandes dificuldades.
* Como o objetivo deste exercício é implementar redes neurais no Keras.
* Vamos começar importando o arquivo CSV bruto usando o Pandas e criando um DataFrame com rótulos de coluna bem organizados.

In [25]:
import pandas as pd

# Definição dos nomes das colunas traduzidos para o português
feature_names =  ['partido', 'bebes-com-deficiencia', 'compartilhamento-custos-projetos-hidricos',  
                  'adocao-resolucao-orcamentaria', 'congelamento-taxas-medicas',  
                  'auxilio-el-salvador', 'grupos-religiosos-escolas',  
                  'proibicao-testes-satelites', 'ajuda-contras-nicaragua',  
                  'missil-mx', 'imigracao', 'reducao-financiamento-combustiveis-sinteticos',  
                  'gastos-educacao', 'direito-processar-superfund', 'legislacao-criminal',  
                  'exportacoes-isentas-impostos', 'lei-administracao-exportacoes-africa-do-sul']

# Carregamento do arquivo CSV, substituindo '?' por NaN e aplicando os novos nomes das colunas
voting_data = pd.read_csv('house-votes-84.data.txt', na_values=['?'], 
                          names=feature_names)

# Exibição das primeiras linhas do DataFrame
voting_data.head()

,partido,bebes-com-deficiencia,compartilhamento-custos-projetos-hidricos,adocao-resolucao-orcamentaria,congelamento-taxas-medicas,auxilio-el-salvador,grupos-religiosos-escolas,proibicao-testes-satelites,ajuda-contras-nicaragua,missil-mx,imigracao,reducao-financiamento-combustiveis-sinteticos,gastos-educacao,direito-processar-superfund,legislacao-criminal,exportacoes-isentas-impostos,lei-administracao-exportacoes-africa-do-sul
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [26]:
voting_data.describe()

,partido,bebes-com-deficiencia,compartilhamento-custos-projetos-hidricos,adocao-resolucao-orcamentaria,congelamento-taxas-medicas,auxilio-el-salvador,grupos-religiosos-escolas,proibicao-testes-satelites,ajuda-contras-nicaragua,missil-mx,imigracao,reducao-financiamento-combustiveis-sinteticos,gastos-educacao,direito-processar-superfund,legislacao-criminal,exportacoes-isentas-impostos,lei-administracao-exportacoes-africa-do-sul
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


* ### Podemos ver que há alguns dados ausentes para lidar aqui; 
* ### alguns políticos se abstiveram em algumas votações ou simplesmente não estavam presentes quando a votação ocorreu. 
* ### Vamos simplesmente remover as linhas com dados ausentes para manter as coisas simples, mas, na prática, seria importante primeiro garantir que essa remoção não introduzisse nenhum tipo de viés na análise (por exemplo, se um partido se abstiver mais do que o outro, isso poderia ser problemático).

In [27]:
voting_data.dropna(inplace=True)
voting_data.describe()

,partido,bebes-com-deficiencia,compartilhamento-custos-projetos-hidricos,adocao-resolucao-orcamentaria,congelamento-taxas-medicas,auxilio-el-salvador,grupos-religiosos-escolas,proibicao-testes-satelites,ajuda-contras-nicaragua,missil-mx,imigracao,reducao-financiamento-combustiveis-sinteticos,gastos-educacao,direito-processar-superfund,legislacao-criminal,exportacoes-isentas-impostos,lei-administracao-exportacoes-africa-do-sul
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


* ### Nossa rede neural precisa de números normalizados, não de strings, para funcionar. Então, vamos substituir todos os "y" por 1 e os "n" por 0, além de representar os partidos como 1 e 0 também.

In [28]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

C:\Users\Hian\AppData\Local\Temp\ipykernel_16372\1551821804.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  voting_data.replace(('y', 'n'), (1, 0), inplace=True)
C:\Users\Hian\AppData\Local\Temp\ipykernel_16372\1551821804.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)


In [29]:
voting_data.head()

,partido,bebes-com-deficiencia,compartilhamento-custos-projetos-hidricos,adocao-resolucao-orcamentaria,congelamento-taxas-medicas,auxilio-el-salvador,grupos-religiosos-escolas,proibicao-testes-satelites,ajuda-contras-nicaragua,missil-mx,imigracao,reducao-financiamento-combustiveis-sinteticos,gastos-educacao,direito-processar-superfund,legislacao-criminal,exportacoes-isentas-impostos,lei-administracao-exportacoes-africa-do-sul
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


### Por fim, vamos extrair as características e os rótulos no formato que o Keras espera.

In [30]:
all_features = voting_data[feature_names].drop('partido', axis=1).values
all_classes = voting_data['partido'].values

* ### Como existem apenas dois partidos, este é um problema binário. Isso também nos poupa o trabalho de representar as classes no formato "one-hot".

In [31]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()

    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))

    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn=create_model, epochs=100)

cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

Epoch 1/100


C:\Users\Hian\AppData\Local\Temp\ipykernel_16372\960805555.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_model, epochs=100)


7/7 [==============================] - 0s 755us/step - loss: 0.6929 - accuracy: 0.4904
Epoch 2/100
7/7 [==============================] - 0s 681us/step - loss: 0.6912 - accuracy: 0.5288
Epoch 3/100
7/7 [==============================] - 0s 838us/step - loss: 0.6887 - accuracy: 0.7500
Epoch 4/100
7/7 [==============================] - 0s 1ms/step - loss: 0.6844 - accuracy: 0.8125
Epoch 5/100
7/7 [==============================] - 0s 946us/step - loss: 0.6774 - accuracy: 0.8606
Epoch 6/100
7/7 [==============================] - 0s 768us/step - loss: 0.6664 - accuracy: 0.8702
Epoch 7/100
7/7 [==============================] - 0s 836us/step - loss: 0.6504 - accuracy: 0.8798
Epoch 8/100
7/7 [==============================] - 0s 871us/step - loss: 0.6279 - accuracy: 0.8846
Epoch 9/100
7/7 [==============================] - 0s 866us/step - loss: 0.6003 - accuracy: 0.8750
Epoch 10/100
7/7 [==============================] - 0s 851us/step - loss: 0.5640 - accuracy: 0.8990
Epoch 11/100
7/7 [=====

0.9438405811786652